In [37]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from PIL import Image
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics
import os
import random
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical

In [38]:
def save_img(imgs,names):
  img_new = Image.new('L',(280,280))
  index = 0
  for i in range(0,280,80):
    for j in range(0,280,80):
      img = imgs[index]
      img = Image.fromarray(img,mode='L')
      img_new.paste(img,(i,j))
      index+=1
  img_new.save(names)

In [39]:
def feature_scale(x):
  x = tf.cast(x,dtype=tf.float32)/255.
#  y = tf.cast(y,dtype=tf.int32)
  return x

In [40]:
#Dim reduct nums
dim_reduce = 10
batch_num = 128
lr = 1e-3

In [50]:
(x,y),(x_test,y_test) = datasets.fashion_mnist.load_data()
data = tf.data.Dataset.from_tensor_slices(x)
data = data.map(feature_scale).shuffle(10000).batch(128)

data_test = tf.data.Dataset.from_tensor_slices(x_test)
data_test = data_test.map(feature_scale).batch(128)

data_iter = iter(data)
samples = next(data_iter)
print(samples[0].shape,samples[1].shape)

(28, 28) (28, 28)


In [51]:
class VAE(keras.Model):
  def __init__(self):
    super(VAE,self).__init__()
    #encoder
    self.fc_layer_1 = layers.Dense(128)
    self.fc_layer_2 = layers.Dense(dim_reduce)
    self.fc_layer_3 = layers.Dense(dim_reduce)
    self.fc_layer_4 = layers.Dense(128)
    self.fc_layer_5 = layers.Dense(784)
    

  def model_encoder(self, x):
    h = tf.nn.relu(self.fc_layer_1(x))
    mean_fc = self.fc_layer_2(h)
    var_fc = self.fc_layer_3(h)
    return mean_fc,var_fc

  def model_decoder(self, z):
    out = tf.nn.relu(self.fc_layer_4(z))
    out = self.fc_layer_5(out)
    return out

  def reparameter(self,mean_x,var_x):
    eps = tf.random.normal(var_x.shape)
    std = tf.exp(var_x)**0.5
    z = mean_x + std*eps
    return z
  
  def call(self, inputs, training=None):
    mean_x,var_x = self.model_encoder(inputs)
    z = self.reparameter(mean_x,var_x)
    x = self.model_decoder(z)
    return x,mean_x,var_x

In [61]:
model = VAE()
model.build(input_shape=(4,784))
optimizer = optimizers.Adam(lr=lr)
model.summary()
model.compile(optimizer = optimizer)

Model: "vae_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             multiple                  100480    
_________________________________________________________________
dense_16 (Dense)             multiple                  1290      
_________________________________________________________________
dense_17 (Dense)             multiple                  1290      
_________________________________________________________________
dense_18 (Dense)             multiple                  1408      
_________________________________________________________________
dense_19 (Dense)             multiple                  101136    
Total params: 205,604
Trainable params: 205,604
Non-trainable params: 0
_________________________________________________________________


In [10]:
!rm -rf img_result
!mkdir img_result

'rm' 不是內部或外部命令、可執行的程式或批次檔。


In [11]:
optimizer = optimizers.Adam(lr=lr)
for i in range(10):
  for step,x in enumerate(data):
    x = tf.reshape(x,[-1,784])
    with tf.GradientTape() as tape:
      logits,mean_x,var_x = model(x)
      loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=x,logits=logits)
      loss = tf.reduce_sum(loss)/x.shape[0]
      kl_div = -0.5*(var_x+1-mean_x**2-tf.exp(var_x))
      kl_div = tf.reduce_sum(kl_div)/x.shape[0]
      
      loss = loss + 1.*kl_div
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    
    if step %100==0:
      print(i,step,'loss:',float(loss),'kl_div:',float(kl_div))
      
  x = next(iter(data_test))
  val_x = tf.reshape(x,[-1,784])
  logits,_,_ = model(val_x)
  x_hat = tf.sigmoid(logits)
  x_hat = tf.reshape(x_hat,[-1,28,28])
  x_hat = x_hat.numpy()*255
  x_hat = x_hat.astype(np.uint8)
  save_img(x_hat,'img_result/AE_img_%d.png'%i)

0 0 loss: 547.6412353515625 kl_div: 1.6040430068969727
0 100 loss: 313.370361328125 kl_div: 14.808939933776855
0 200 loss: 270.4798583984375 kl_div: 15.425350189208984
0 300 loss: 271.3494873046875 kl_div: 14.688753128051758
0 400 loss: 254.52890014648438 kl_div: 15.055817604064941
1 0 loss: 268.33538818359375 kl_div: 13.637407302856445
1 100 loss: 261.97802734375 kl_div: 13.590749740600586
1 200 loss: 258.56719970703125 kl_div: 14.495540618896484
1 300 loss: 257.16766357421875 kl_div: 14.42913818359375
1 400 loss: 256.2521667480469 kl_div: 14.208840370178223
2 0 loss: 253.90521240234375 kl_div: 14.05722713470459
2 100 loss: 262.0010986328125 kl_div: 14.271929740905762
2 200 loss: 246.49794006347656 kl_div: 13.743783950805664
2 300 loss: 262.81915283203125 kl_div: 14.200222969055176
2 400 loss: 259.02520751953125 kl_div: 14.237686157226562
3 0 loss: 255.36782836914062 kl_div: 14.131688117980957
3 100 loss: 250.78656005859375 kl_div: 14.587200164794922
3 200 loss: 249.3467254638672 kl_d